In [ ]:
import numpy as np
import pandas as pd
import linearsolve as ls
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# Class 17: Exogenous Shocks in the New-Keynesian Model

In this notebook, we will compute impulse responses to exogenous shocks in the new-Keynesian model and interpret the results.

### Households

\begin{align}
C_t^{-\sigma} & = \beta (1+r_t)E_t C_{t+1}^{-\sigma}\\
W_t & = \varphi (1-L_t)^{-\eta} C^{\sigma}\\
C_t^{-\sigma} & = \beta E_t C_{t+1}^{-\sigma}\left( R_{t+1} + 1-\delta \right)\\
\end{align}

### Firms, Production, and Aggregate Supply

\begin{align}
W_t & = (1-\alpha) A_t K_{t}^{\alpha}L_t^{-\alpha}/X_t\\
R_t & = \alpha A_t K_{t}^{\alpha-1}L_t^{1-\alpha}/X_t\\
Y_t & = A_t K_t^{\alpha}L_t^{1-\alpha}\\
\pi_t & = \beta E_t\pi_{t+1} - \kappa \log(X_t/\bar{X}) + u_t\\
Y_t & = C_t + I_t + G_t
\end{align}

### Monetary Policy

\begin{align}
i_{t} & = \phi_{\pi}\pi_t + \phi_{y} \log (Y_t/\bar{Y}) + v_t\\
\end{align}

### Evolution of State Variables


\begin{align}
K_{t+1} & = I_{t} + (1-\delta) K_t\\
\log A_{t+1} & = \rho_a \log A_{t} + \epsilon^a_{t+1}\\
\log (G_{t+1}/\bar{G}) & = \rho_g \log (G_{t}/\bar{G}) + \epsilon^g_{t+1}\\
u_{t+1} & = \rho_u u_{t} + \epsilon^u_{t+1}\\
v_{t+1} & = \rho_v v_{t} + \epsilon^v_{t+1}.
\end{align}

### Fisher Equation

\begin{align}
i_{t} & = r_{t} + E_t \pi_{t+1}\\
\end{align}

The goal is to compute impulse responses in the model to all three exogenous shocks. We will use the following parameterization:

| $\bar{y}$ | $\beta$ | $\bar{r}$    | $\kappa$ | $\pi^T$ | $\phi_{\pi}$ | $\phi_y$ | $\rho_g$ | $\rho_u$ | $\rho_v$ | 
|-----------|---------|--------------|----------|---------|--------------|----------|----------|----------|----------|
| 0         | 0.995   | $-\log\beta$ | 0.1      | 0.02/4  | 1.5          | 0.5/4    | 0.5      | 0.5      | 0.5      |

In [ ]:


# values = pd.Series(guess,index=varNames)
# print(values)




In [ ]:
# Define a function that evaluates the equilibrium conditions of the model solved for zero.


    # Parameters


    # Current variables


    # Forward variables


    # Euler equation


    # Labor supply


    # Capital supply


    # Labor demand


    # Capital demand


    # Production


    # Market clearing


    # Monetary policy


    # Fisher equation


    # Phillips curve
#     phillips_curve = (1-p.beta) + p.beta*(fwd.pi) - p.kappa*np.log(cur.x/p.x_bar)  + np.log(cur.u) - cur.pi

    # Capital evolution


    # Government purchases process


    # TFP process


    # Inflation shock process


    # Monetary policy shock process


    # Stack equilibrium conditions into a numpy array


# Initialize the model into a variable named 'nk_model'


# Compute the steady state numerically using .compute_ss() method of nk_model




In [ ]:

# model.impulse(t0=5,shocks=[0,0.01,0.01,0.01/4,0.01/4,0.01/4],percent=True,T=26)

## Solution

In [ ]:
# Create a variable called 'parameters' that stores the model parameter values in a Pandas Series. CELL PROVIDED
parameters = pd.Series()
parameters['y_bar'] = 0
parameters['beta'] = 0.995
parameters['r_bar'] = -np.log(parameters.beta)
parameters['kappa'] = 0.1
parameters['pi_T'] = 0.02/4
parameters['phi_pi'] = 1.5
parameters['phi_y'] = 0.5/4
parameters['rho_g'] = 0.5
parameters['rho_u'] = 0.5
parameters['rho_v'] = 0.5

# Create variable called 'varNames' that stores the variable names in a list with state variables ordered first
varNames = ['g','u','v','y','pi','i','r']

# Create variable called 'shockNames' that stores an exogenous shock name for each state variable.
shockNames = ['e_g','e_u','e_v']

# Define a function that evaluates the equilibrium conditions of the model solved for zero.
def equilibrium_equations(variables_forward,variables_current,parameters):
    
    # Parameters
    p = parameters
    
    # Current variables
    cur = variables_current
    
    # Forward variables
    fwd = variables_forward

    # IS equation
    is_equation = fwd.y - (cur.r -p.r_bar) + cur.g - cur.y
    
    # Fisher_equation
    fisher_equation = cur.r + fwd.pi - cur.i
    
    # Monetary policy
    monetary_policy = p.r_bar + p.pi_T + p.phi_pi*(cur.pi - p.pi_T) + p.phi_y*cur.y + cur.v - cur.i
    
    # Phillips curve
    phillips_curve = p.beta*(fwd.pi- p.pi_T) + p.kappa*cur.y + cur.u - (cur.pi-p.pi_T)
    
    # Demand process
    demand_process = p.rho_g*cur.g - fwd.g
    
    # Monetary policy process
    monetary_policy_process = p.rho_v*cur.v - fwd.v
    
    # Inflation process
    inflation_process = p.rho_u*cur.u - fwd.u
    
    
    # Stack equilibrium conditions into a numpy array
    return np.array([
            is_equation,
            fisher_equation,
            monetary_policy,
            phillips_curve,
            demand_process,
            monetary_policy_process,
            inflation_process
        ])

# Initialize the model into a variable named 'nk_model'
nk_model = ls.model(equations = equilibrium_equations,
                 nstates=3,
                 varNames=varNames,
                 shockNames=shockNames,
                 parameters = parameters)


# Compute the steady state numerically using .compute_ss() method of nk_model
guess = [k,a,g,y,c,i,l,x,pi,real,nominal_rate,w,r]
nk_model.compute_ss(guess)

In [ ]:
# Find the log-linear approximation around the non-stochastic steady state and solve using .approximate_and_solve() method of nk_model
# set argumement 'loglinear' to False


# Print the solution to the model


**Questions** 

1. Will a positive shock to $g_t$ increase or decrease $y_t$?
2. Will a positive shock to $g_t$ increase or decrease $\pi_t$?
3. Will a positive shock to $u_t$ increase or decrease $y_t$?
4. Will a positive shock to $u_t$ increase or decrease $\pi_t$?

**Answers**

1.  

2.  

1.  

4.  

## Impulse Responses

Compute a 21 period impulse response of the model's variables to a:

* 0.01 unit shock to $g_t$
* 0.01/4 unit shock to $u_t$
* 0.01/4 unit shock to $v_t$

In [ ]:
# Compute impulse responses


### Demand Shock

Plot the computed impulses responses of the nominal_rate interest rate, the real interest rate, output, and inflation to the 0.01 unit demand shock. Express inflation and interest rates in *annualized* (e.g., multiplied by 4) terms.

In [ ]:
# Create figure. PROVIDED
fig = plt.figure(figsize=(12,8))

# Create upper-left axis. PROVIDED
ax1 = fig.add_subplot(2,2,1)


# Create upper-right axis. PROVIDED
ax2 = fig.add_subplot(2,2,2)


# Create lower-left axis. PROVIDED
ax3 = fig.add_subplot(2,2,3)


# Create lower-right axis. PROVIDED
ax4 = fig.add_subplot(2,2,4)




### Inflation Shock

Plot the computed impulses responses of the nominal_rate interest rate, the real interest rate, output, and inflation to the 0.01/4 unit inflation shock. Express inflation and interest rates in *annualized* (e.g., multiplied by 4) terms.

In [ ]:
# Create figure. PROVIDED
fig = plt.figure(figsize=(12,8))

# Create upper-left axis. PROVIDED
ax1 = fig.add_subplot(2,2,1)


# Create upper-right axis. PROVIDED
ax2 = fig.add_subplot(2,2,2)


# Create lower-left axis. PROVIDED
ax3 = fig.add_subplot(2,2,3)


# Create lower-right axis. PROVIDED
ax4 = fig.add_subplot(2,2,4)




### Monetary Policy Shock

Plot the computed impulses responses of the nominal_rate interest rate, the real interest rate, output, and inflation to the 0.01/4 unit monetary policy shock. Express inflation and interest rates in *annualized* (e.g., multiplied by 4) terms.

In [ ]:
# Create figure. PROVIDED
fig = plt.figure(figsize=(12,8))

# Create upper-left axis. PROVIDED
ax1 = fig.add_subplot(2,2,1)


# Create upper-right axis. PROVIDED
ax2 = fig.add_subplot(2,2,2)


# Create lower-left axis. PROVIDED
ax3 = fig.add_subplot(2,2,3)


# Create lower-right axis. PROVIDED
ax4 = fig.add_subplot(2,2,4)


